In [ ]:
import torch
import torchvision

import torch.nn as nn
from torchvision import transforms

In [ ]:
transform = transforms.Compose([transforms.Resize((32, 32)),
                                transforms.ToTensor()])

train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transform,  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=100, 
                                          shuffle=False)

<img src="https://www.researchgate.net/profile/Vladimir-Golovko-5/publication/313808170/figure/fig3/AS:552880910618630@1508828489678/Architecture-of-LeNet-5.png">

## LeNet-5 на PyTorch

In [ ]:
class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1)
        self.conv_layer2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)
        self.conv_layer3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1)
        self.pooling_layer1 = nn.AvgPool2d(kernel_size=2)
        self.pooling_layer2 = nn.AvgPool2d(kernel_size=2)
        
        self.linear_layer1 = nn.Linear(in_features=120, out_features=84)
        self.linear_layer2 = nn.Linear(in_features=84, out_features=num_classes)
        
        self.tanh = nn.Tanh()
        
    def forward(self, inputs):
        output_1 = self.tanh(self.conv_layer1(inputs))
        output_2 = self.pooling_layer1(output_1)
        output_3 = self.tanh(self.conv_layer2(output_2))
        output_4 = self.pooling_layer2(output_3)
        output_5 = self.tanh(self.conv_layer3(output_4))
        output_6 = torch.flatten(output_5, 1)
        
        output_7 = self.tanh(self.linear_layer1(output_6))
        output = self.linear_layer2(output_7)
        
        return output

In [ ]:
model = LeNet5(10)
print(model)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
torch.manual_seed(0)

total_step = len(train_loader)
for epoch in range(5):
    for i, (images, labels) in enumerate(train_loader): 

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, 5, i+1, total_step, loss.item()))

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy: {} %'.format(100 * correct / total))

## LeNet-5 на Keras

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense

from keras.utils import to_categorical

In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=6, kernel_size=(5, 5), activation='tanh', input_shape=(28, 28, 1), 
                 padding='same'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation='tanh'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Flatten())

model.add(Dense(units=120, activation='tanh'))
model.add(Dense(units=84, activation='tanh'))
model.add(Dense(units=10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=100, epochs=5, validation_data=(X_test, y_test))

### Задание

Реализовать и обучить в PyTorch и Keras следующую архитектуру:

<img src="https://www.easy-tensorflow.com/images/cnntext.png">